In [1]:
import os
import urllib.request
import zipfile
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import Callback

In [2]:
data_url = "https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip"
urllib.request.urlretrieve(data_url, "cats_and_dogs.zip")
local_file = "cats_and_dogs.zip"
zip_ref = zipfile.ZipFile(local_file, "r")
zip_ref.extractall("data/")
zip_ref.close()

In [3]:
BASE_DIR = "/content/data/cats_and_dogs_filtered"
train_dir = os.path.join(BASE_DIR, "train")
validation_dir = os.path.join(BASE_DIR, "validation")

In [4]:
print(len(os.listdir(os.path.join(train_dir,"cats"))))
print(len(os.listdir(os.path.join(train_dir,"dogs"))))

1000
1000


In [5]:
print(len(os.listdir(os.path.join(validation_dir,"cats"))))
print(len(os.listdir(os.path.join(validation_dir,"dogs"))))

500
500


In [6]:
train_datagen = ImageDataGenerator(
    rescale=1/255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
)

validation_datagen = ImageDataGenerator(
    rescale=1/255
)

In [7]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [8]:
class super_callback(Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') >=0.83 and logs.get('val_accuracy') >= 0.83:
          print(epoch)
          self.model.stop_training = True

In [9]:
super_cb = super_callback()

In [15]:
model = tf.keras.models.Sequential(
        [
            tf.keras.layers.Conv2D(
                16, (3, 3), activation="relu", input_shape=(150, 150, 3), padding='same', kernel_initializer='he_uniform'
            ),
            tf.keras.layers.MaxPooling2D(2, 2),
            tf.keras.layers.Conv2D(32, (3, 3), activation="relu", padding='same', kernel_initializer='he_uniform'),
            tf.keras.layers.MaxPooling2D(2, 2),
            tf.keras.layers.Conv2D(32, (3, 3), activation="relu", padding='same', kernel_initializer='he_uniform'),
            tf.keras.layers.MaxPooling2D(2, 2),
            tf.keras.layers.Conv2D(64, (3, 3), activation="relu", padding='same', kernel_initializer='he_uniform'),
            tf.keras.layers.MaxPooling2D(2, 2),
            tf.keras.layers.Conv2D(64, (3, 3), activation="relu", padding='same', kernel_initializer='he_uniform'),
            tf.keras.layers.MaxPooling2D(2, 2),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(256, activation="relu"),
            tf.keras.layers.Dense(128, activation="relu"),
            tf.keras.layers.Dropout(0.4),
            tf.keras.layers.Dense(1, activation="sigmoid"),
        ]
    )

In [16]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [17]:
model.fit(
        train_generator,
        epochs=50,
        verbose=0,
        validation_data=validation_generator,
        callbacks=[super_cb]
)

42


In [18]:
model.evaluate(train_generator)

100/100 [==============================] - 14s 136ms/step - loss: 0.3208 - accuracy: 0.8580


[0.3207674026489258, 0.8579999804496765]

In [19]:
model.evaluate(validation_generator)

50/50 [==============================] - 3s 54ms/step - loss: 0.4189 - accuracy: 0.8370


[0.4188588261604309, 0.8370000123977661]